In [1]:
# run_single_profile_with_tokens.py

import os
import logging
import pandas as pd
from schema import parser
from prompt import get_prompt
from llm import get_remote_llm
from config import PROFILES_CSV, QUESTIONS_TXT, LLAMA_MODEL

from transformers import AutoTokenizer

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 1) load the tokenizer for exactly your model
tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL)

def count_tokens(text: str) -> int:
    # count with special tokens so it matches generation call
    return len(tokenizer.encode(text, add_special_tokens=True))

def format_profile(row: pd.Series) -> str:
    return "\n".join(f"{col}: {val}" for col, val in row.items())

def clean_answer(raw: str, question: str) -> str:
    idx = raw.lower().find(question.lower())
    if idx != -1:
        raw = raw[idx + len(question):]
    return raw.lstrip(" :–—\n\t")

def run_one_profile(profile_idx: int = 0):
    # — load one profile
    df = pd.read_csv(PROFILES_CSV)
    row = df.iloc[profile_idx]
    student_name = row["name"]
    student_id   = profile_idx

    # — load questions
    with open(QUESTIONS_TXT) as f:
        questions = [q.strip() for q in f if q.strip()]

    # — build your PromptTemplate and chain
    prompt_tpl = get_prompt()
    format_instructions = parser.get_format_instructions()
    llm = get_remote_llm()
    chain = prompt_tpl | llm

    # accumulators
    sum_input_tokens = 0
    sum_output_tokens = 0
    sum_total_tokens = 0

    for q in questions:
        # 1) render the exact prompt text
        prompt_inputs = {
            "student_name": student_name,
            "student_id":   student_id,
            "question":     q,
            "format_instructions": format_instructions
        }
        prompt_text = prompt_tpl.format(**prompt_inputs)

        # 2) count input tokens
        in_toks = count_tokens(prompt_text)

        # 3) call the chain
        resp = chain.invoke(prompt_inputs)
        raw = resp[0] if isinstance(resp, list) else resp
        ans = clean_answer(raw, q)

        # 4) count output tokens
        out_toks = count_tokens(ans)

        total_toks = in_toks + out_toks
        sum_input_tokens  += in_toks
        sum_output_tokens += out_toks
        sum_total_tokens  += total_toks

        print(f"Q: {q[:30]}… → in={in_toks}, out={out_toks}, total={total_toks}")

    n = len(questions)
    avg_per_q = sum_total_tokens / n
    total_calls = 1_000 * 40
    est_tokens  = avg_per_q * total_calls

    print("\n=== TOKEN USAGE SUMMARY ===")
    print(f"Questions run          : {n}")
    print(f"Sum input tokens       : {sum_input_tokens}")
    print(f"Sum output tokens      : {sum_output_tokens}")
    print(f"Sum total tokens       : {sum_total_tokens}")
    print(f"Average tokens / query : {avg_per_q:.2f}")
    print(f"\nExtrapolation to 1 000 profiles × 40 questions:")
    print(f"  Total calls           : {total_calls}")
    print(f"  Estimated tokens      : {est_tokens:,.0f}")

run_one_profile(0)


c:\micromamba\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\aryan\Desktop\Personalized_Education\Personalized_Education\Finetuned_Answers\llm.py:39: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  return HuggingFaceEndpoint(
                    input_key was transferred to model_kwargs.
                    Please make sure that input_key is what you intended.
                    output_key was transferred to model_kwargs.
                    Please make sure th

Q: Explain the human digestive sy… → in=223, out=513, total=736


c:\micromamba\envs\langchain\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Q: How does reproduction occur in… → in=223, out=84, total=307


c:\micromamba\envs\langchain\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Q: What are acids and bases? Give… → in=228, out=409, total=637


c:\micromamba\envs\langchain\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Q: What is the function of the re… → in=227, out=513, total=740

=== TOKEN USAGE SUMMARY ===
Questions run          : 4
Sum input tokens       : 901
Sum output tokens      : 1519
Sum total tokens       : 2420
Average tokens / query : 605.00

Extrapolation to 1 000 profiles × 40 questions:
  Total calls           : 40000
  Estimated tokens      : 24,200,000
